In [13]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [14]:
def getCovidData(category):
    if category == "confirmed":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    
    elif category == "recovered":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
        
    elif category == "deaths":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

    soup = BeautifulSoup(response.content, 'html.parser')
    html_table=soup.find('table')
    all_rows=html_table.find_all('tr')
    jh_data_list=[]
    for pos,rows in enumerate(all_rows):
        if pos==0:
            header_list = [each_col.get_text(strip=True) for each_col in rows.find_all('th')]
        else:
            col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
            jh_data_list.append(col_list)
    header_list.insert(0,'index')
    jh_data_df=pd.DataFrame(jh_data_list,index=None).dropna()
    jh_data_df.columns=header_list
    jh_data_df.drop(columns=['index','Lat','Long'],axis=1)

    time_idx=jh_data_df.columns[5:]
    
    df_plot = pd.DataFrame({'date':time_idx})

    country_list=jh_data_df['Country/Region']
    for each in country_list:
        df_plot[each]=np.array(jh_data_df[jh_data_df['Country/Region']==each].iloc[:,5::].sum(axis=0))

    time_idx=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] # convert to datetime
    time_str=[each.strftime('%Y-%m-%d') for each in time_idx] # convert back to date ISO norm (str)

    df_plot['date']=time_idx
    df_plot[df_plot.columns[1:]] = df_plot[df_plot.columns[1:]].astype(int)
    
    return df_plot


In [15]:
df_confirmed = getCovidData("confirmed")
df_recovered = getCovidData("recovered")
df_deaths = getCovidData("deaths")

In [16]:
df_confirmed.to_csv('../data/processed/COVID_small_flat_table_confirmed.csv',sep=';',index=False)
df_recovered.to_csv('../data/processed/COVID_small_flat_table_recovered.csv',sep=';',index=False)
df_deaths.to_csv('../data/processed/COVID_small_flat_table_deaths.csv',sep=';',index=False)

In [20]:
df_confirmed.head()

,date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,3000010,...,0,0,0,0,0,2,0,0,0,0
